In [1]:
import datetime
import pandas as pd
from calendar import monthrange
import numpy as np
from pandas.tseries.offsets import MonthEnd
import csv
import warnings; warnings.filterwarnings('ignore')

In [1]:
path = '\\\\svrau055csm00\\OTSPC SHRSV\\SHARE\\Shared Services\\Group Services\\Group Surveys\\Placeholder_BGL_File_Sharing\\RJN\\Vdata\\'

LeadFile = pd.read_csv(path+'Dummy Data.csv')
Info = pd.read_csv(path+'Info.csv')

SingleQ = list(Info['Qns'][(Info['Type'] == "Single")])
NumericQ = list(Info['Qns'][(Info['Type'] == "Numeric")])
OpenQ = list(Info['Qns'][(Info['Type'] == "Open")])

MultidfInf = Info[(Info['Type'] == "Multi")]
MultiQ = list(set(MultidfInf['QID']))
MultiQns = list(set(MultidfInf['Qns']))

GrddfInf = Info[(Info['Type'] == "Grid")]
GrdQ = list(set(GrddfInf['QID']))

def basecheck(Qtype):
    InfDum = Info.loc[(Info['Base Condition'].notnull()) & (Info['Type'] == Qtype)]
    if len(InfDum) > 0:
        new = InfDum['Base Condition'].str.split("=", n = 1, expand = True)
        InfDum["QIDdum"]= new[0].str.strip()
        InfDum["Conddum"]= new[1].str.strip()
        InfDum.drop_duplicates(subset ="QID", inplace = True)                
        BSC = list(LeadFile['RespID'][(LeadFile[InfDum['QIDdum'].item()] \
                                       == int(InfDum['Conddum'].item())) & (~LeadFile[Q].notnull())])
        Info.loc[Info['QID'] == Q, 'Base check'] = str(BSC)  

# Code for checking blanks in single Question
for Q in SingleQ:
    BC = list(LeadFile['RespID'][~LeadFile[Q].notnull()])
    Info.loc[Info['Qns'] == Q, 'Blank ids'] = str(BC)

for Q in MultiQns:
    MultidfInfDum = MultidfInf.loc[MultidfInf['Base Condition'].notnull()]
    if len(MultidfInfDum) > 0:
        new = MultidfInfDum['Masking'].str.split("=", n = 1, expand = True)
        MultidfInfDum["QIDdum"]= new[0].str.strip()
        MultidfInfDum["Conddum"]= new[1].str.strip()
        
# Code for checking blanks and range in Numeric Question
for Q in NumericQ:
    BC = list(LeadFile['RespID'][~LeadFile[Q].notnull()])
    Info.loc[Info['Qns'] == Q, 'Blank ids'] = str(BC)
    RC = list(LeadFile['RespID'][~LeadFile[Q].between(int(Info['Min'][Info['Qns'] == Q]),\
                                                      int(Info['Max'][Info['Qns'] == Q]))])
    Info.loc[Info['Qns'] == Q, 'Range Check'] = str(RC)
    basecheck("Numeric")
    
# Code for checking blanks and length in Open Question
for Q in OpenQ:
    BC = list(LeadFile['RespID'][~LeadFile[Q].notnull()])
    Info.loc[Info['Qns'] == Q, 'Blank ids'] = str(BC)
    LC = list(LeadFile['RespID'][~LeadFile[Q].str.len().between(int(Info['Min'][Info['Qns'] == Q]),\
                                                                int(Info['Max'][Info['Qns'] == Q]))])
    Info.loc[Info['Qns'] == Q, 'Range Check'] = str(LC)
    basecheck("Open")

# Code for checking blanks in Multi Question
for Q in MultiQ:    
    tempQ = list(MultidfInf['Qns'][MultidfInf['QID'] == Q])
    Qarr = list(tempQ)
        
    LeadFile[Q] = LeadFile[Qarr].sum(axis=1).round() 
    BC = list(LeadFile['RespID'][LeadFile[Q] == 0])
    Info.loc[Info['QID'] == Q, 'Blank ids'] = str(BC)
    
    MultidfInfDum = MultidfInf.loc[MultidfInf['Base Condition'].notnull()]
    if len(MultidfInfDum) > 0:
        new = MultidfInfDum['Base Condition'].str.split("=", n = 1, expand = True)
        MultidfInfDum["QIDdum"]= new[0].str.strip()
        MultidfInfDum["Conddum"]= new[1].str.strip()
        MultidfInfDum.drop_duplicates(subset ="QID", inplace = True)        
        BSC = list(LeadFile['RespID'][(LeadFile[MultidfInfDum['QIDdum'][MultidfInfDum['QID'] == Q].item()] \
                                       == int(MultidfInfDum['Conddum'][MultidfInfDum['QID'] == Q].item())) & (LeadFile[Q] == 0)])
        Info.loc[Info['QID'] == Q, 'Base check'] = str(BSC)    

    LeadFile[Q] = LeadFile[Qarr].count(axis=1) 
    BC = list(LeadFile['RespID'][(LeadFile[Q] != len(Qarr)) & (LeadFile[Q] > 0)])
    Info.loc[Info['QID'] == Q, 'Missing options'] = str(BC)

# Code for checking blanks in Grid Question
for Q in GrdQ:    
    tempQ = list(GrddfInf['Qns'][GrddfInf['QID'] == Q])
    Qarr = list(tempQ)

    LeadFile[Q] = LeadFile[Qarr].sum(axis=1).round() 
    BC = list(LeadFile['RespID'][LeadFile[Q] == 0])
    Info.loc[Info['QID'] == Q, 'Blank ids'] = str(BC)
    
    GrddfInfDum = GrddfInf.loc[GrddfInf['Base Condition'].notnull()]
    if len(GrddfInfDum) > 0:
        new = GrddfInfDum['Base Condition'].str.split("=", n = 1, expand = True)
        GrddfInfDum["QIDdum"]= new[0].str.strip()
        GrddfInfDum["Conddum"]= new[1].str.strip()
        GrddfInfDum.drop_duplicates(subset ="QID", inplace = True)                
        BSC = list(LeadFile['RespID'][(LeadFile[GrddfInfDum['QIDdum'].item()] \
                                       == int(GrddfInfDum['Conddum'].item())) & (LeadFile[Q] == 0)])
        Info.loc[Info['QID'] == Q, 'Base check'] = str(BSC) 

    LeadFile[Q] = LeadFile[Qarr].count(axis=1) 
    BC = list(LeadFile['RespID'][(LeadFile[Q] != len(Qarr)) & (LeadFile[Q] > 0)])
    Info.loc[Info['QID'] == Q, 'Missing options'] = str(BC)

    

# print(LeadFile)
# print("\n")
# print(Info[['QID','Type','Blank ids','Range Check','Missing options']])
# print("\n")
dummyinfo = Info[['QID','Type','Blank ids','Base check','Range Check','Missing options']]
dummyinfo.drop_duplicates(subset ="QID", inplace = True) 
print(dummyinfo)

NameError: name 'pd' is not defined

In [ ]:
#Requirements : 
#check in grid/Multi if all are answered - Missing options column created
#base condition check
#answer option masking